In [3]:
import torch
from torch import nn
from d2l import torch as d2l

In [16]:
def corr2d(X,K):
    h,w=K.shape
    Y=torch.zeros((X.shape[0]-h+1,X.shape[1]-w+1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i,j]=(X[i:i+h,j:j+w]*K).sum()
    return Y

In [9]:
X=torch.tensor([[0.0,1.0,2.0],[3.0,4.0,5.0],[6.0,7.0,8.0]])
K=torch.tensor([[0.0,1.0],[2.0,3.0]])
corr2d(X,K)

tensor([[19., 25.],
        [37., 43.]])

In [18]:
X=torch.ones((6,8))
X[:,2:6]=0
K=torch.tensor([[1.0,-1.0]])
Y=corr2d(X,K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

In [22]:
conv2d=nn.LazyConv2d(1,kernel_size=(1,2),bias=False)#1是输出通道数
X=X.reshape((1,1,6,8))
Y=Y.reshape((1,1,6,7))
lr=1e-2
for i in range(20):
    Y_hat=conv2d(X)
    l=(Y_hat-Y)**2
    conv2d.zero_grad()
    l.sum().backward()
    conv2d.weight.data[:]-=lr*conv2d.weight.grad
    print(f"epoch{i+1},loss{l.sum():.3f}")
conv2d.weight.data

epoch1,loss13.023
epoch2,loss10.071
epoch3,loss7.796
epoch4,loss6.037
epoch5,loss4.675
epoch6,loss3.620
epoch7,loss2.804
epoch8,loss2.171
epoch9,loss1.681
epoch10,loss1.302
epoch11,loss1.008
epoch12,loss0.781
epoch13,loss0.605
epoch14,loss0.468
epoch15,loss0.363
epoch16,loss0.281
epoch17,loss0.217
epoch18,loss0.168
epoch19,loss0.130
epoch20,loss0.101


tensor([[[[ 0.9193, -0.9193]]]])

In [ ]:
## 

In [24]:
import torch
from torch import nn

def comp_conv2d(conv2d, X):
    #批量大小和通道数都是1
    X = X.reshape((1, 1) + X.shape)
    Y = conv2d(X)
    return Y.reshape(Y.shape[2:])
conv2d=nn.Conv2d(1, 1, kernel_size=3, padding=1)
X=torch.rand(size=(8, 8))
comp_conv2d(conv2d, X).shape
#8-3+1+1*2

torch.Size([8, 8])

In [30]:
conv2d=nn.Conv2d(1, 1, kernel_size=(5, 3), padding=(2, 1))
comp_conv2d(conv2d, X).shape
#8-5+2*2+1
#8-3+1*2+1

torch.Size([8, 8])

In [31]:
conv2d=nn.Conv2d(1, 1, kernel_size=(3, 5), padding=(0, 1), stride=(3, 4))
comp_conv2d(conv2d, X).shape
#(8-3-0+3)/3=2
#(8-5+1*2+4)/4=2
#记得不用加1

torch.Size([2, 2])

## 多通道

In [33]:
def corr2d_multi_in(X, K):
    #先遍历通道维度，再把它们加在一起
    return sum(d2l.corr2d(x, k) for x, k in zip(X, K))

In [34]:
X=torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
               [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K=torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])

corr2d_multi_in(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])

In [36]:
def corr2d_multi_in_out(X, K):
    #stack将所有结果都叠加在一起，0是维度
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)
K=torch.stack((K, K + 1, K + 2), 0)
corr2d_multi_in_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

## pooling

In [38]:
def pool2d(X, pool_size, mode='max'):
    p_h, p_w = pool_size
    Y = torch.zeros((X.shape[0]-p_h + 1, X.shape[1]- p_w+ 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            if mode == 'max':
                Y[i, j]=X[i: i + p_h, j: j + p_w].max()
            elif mode == 'avg':
                Y[i, j]=X[i: i + p_h, j: j + p_w].mean()
    return Y
X=torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
print(pool2d(X,(2,2)))
print(pool2d(X,(2,2),mode='avg'))

tensor([[4., 5.],
        [7., 8.]])
tensor([[2., 3.],
        [5., 6.]])


In [42]:
#含有stride和padding
X=torch.arange(16, dtype=torch.float32).reshape((1, 1, 4, 4))
pool2d=nn.MaxPool2d((2, 3), stride=(2, 3), padding=(0, 1))
#(4-2+0+2)/2=2
#(4-3+1*2+3)/3=2
pool2d(X).shape

torch.Size([1, 1, 2, 2])